In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd

from glob import glob
from tensorflow.keras.layers import Input, Flatten, Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [3]:
input_dir = "signature_data"
maxlen = 200  # Maximum number of strokes
labels = os.listdir(input_dir)

In [4]:
def features_from_txt(filename):
    folder = os.path.basename(os.path.dirname(filename))
    data = np.loadtxt(filename, skiprows=1)
    # Column-wise min-max scaling
    data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
    if len(data) < maxlen:
        pad = maxlen - len(data)
        data = np.pad(data, ((0, pad), (0, 0)))
    data = data[:maxlen, :]
    label = to_categorical(labels.index(folder), num_classes=len(labels))
    return (data, label)

In [5]:
def generate_data(input_dir, shuffle=False):
    files = glob(os.path.join(input_dir, "*", "*"))
    random.shuffle(files)
    n = len(files)
    def fetch():
        i = 0
        while True:
            if shuffle:
                file = random.choice(files)
            else:
                file = files[i]
                i = (i+1)%n
            yield features_from_txt(file)
    return fetch

In [7]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(generate_data(input_dir), output_types=(tf.float64, tf.uint8), output_shapes=((200, 7), (20,)))
dataset = dataset.batch(batch_size)
for d in dataset.take(1):
    print(d[0].shape, d[1].shape)

(32, 200, 7) (32, 20)


In [8]:
def signature_model():
    x1 = Input(shape=(maxlen, 7), name='signature_input')
    x = LSTM(128, return_sequences=True)(x1)
    x = LSTM(64)(x)
    x = Dense(20, activation='softmax',name='signature_output')(x)
    return Model(inputs=x1, outputs=x)

In [9]:
model = signature_model()

epochs = 20
lr = 0.001
save_model_as = "signature_epochs{}_lr{}_batch{}"
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(dataset, epochs=epochs, steps_per_epoch=20)
model.save(save_model_as.format(epochs, lr, batch_size))

Train for 20 steps
Epoch 1/20
20/20 [==============================] - 13s 627ms/step - loss: 2.7516 - accuracy: 0.1562
Epoch 2/20
20/20 [==============================] - 3s 147ms/step - loss: 2.2411 - accuracy: 0.3031
Epoch 3/20
20/20 [==============================] - 3s 146ms/step - loss: 1.7803 - accuracy: 0.4109
Epoch 4/20
20/20 [==============================] - 3s 148ms/step - loss: 1.3661 - accuracy: 0.5547
Epoch 5/20
20/20 [==============================] - 3s 175ms/step - loss: 1.3657 - accuracy: 0.5141
Epoch 6/20
20/20 [==============================] - 3s 146ms/step - loss: 1.0448 - accuracy: 0.6469
Epoch 7/20
20/20 [==============================] - 3s 153ms/step - loss: 0.8682 - accuracy: 0.7063
Epoch 8/20
20/20 [==============================] - 3s 151ms/step - loss: 0.8449 - accuracy: 0.6859
Epoch 9/20
20/20 [==============================] - 3s 162ms/step - loss: 0.7803 - accuracy: 0.7063
Epoch 10/20
20/20 [==============================] - 3s 162ms/step - loss: 0.732